<a href="https://colab.research.google.com/github/italomarcelogit/auto-sklearn-vendas/blob/main/auto_sklearn_vendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab - Preparando ambiente de execução**

In [ ]:
import warnings
warnings.filterwarnings("ignore")
!pip install --upgrade -q pip
!pip install --upgrade -q scipy==1.7.1
!pip install -q auto-sklearn

**IMPORTANTE:**
Após as importações e instalações anteriores, reinicie o ambiente

In [3]:
import pandas as pd
import numpy as np
import sklearn.metrics
from datetime import datetime
from time import time
import plotly.graph_objects as go
import autosklearn.regression

# **Dataset**
*A fonte dos dados foi criada com valores fictícios e tem o objetivo de facilitar a compreensão dos dados são apresentados*

*Os dados abaixo contém a varíavel ds que determina o dado temporal, neste caso está com uma data por mês/ano, onde y é a somatória de todas as vendas y1 a y6*

In [4]:
url = 'https://raw.githubusercontent.com/italomarcelogit/keras-regressor/main/dados.csv'
df = pd.read_csv(url, sep=';')
df.head(2)

,ds,y,y1,y2,y3,y4,y5,y6
0,2000-01-01,750.0,100.0,100.0,150.0,150.0,130.0,120.0
1,2000-02-01,750.0,100.0,100.0,150.0,150.0,130.0,120.0


In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['ds'], y=df['y'],
                name='Realizado', marker_color='lightsalmon'))

fig.update_layout(title='Fluxo de Vendas Realizadas',
                  xaxis_title='Período',
                  yaxis_title='Total (R$)',
                  template='plotly_white',
                  xaxis = dict(tickangle=80)
                  )
fig.show()

# **Pre-Processamento**

In [9]:
df2 = df[['ds', 'y']].copy()
# diaAno == day of year from ds
df2['diaAno'] = df2.ds.apply(lambda x: pd.to_datetime([x]).dayofyear[0])
# ano == year from ds
df2['ano'] = df2.ds.apply(lambda x: pd.to_datetime([x]).year[0])
df2.pop('ds')
df2.head()

,y,diaAno,ano
0,750.0,1,2000
1,750.0,32,2000
2,1000.0,61,2000
3,1000.0,92,2000
4,1000.0,122,2000


In [10]:
df2.ano.min(), df2.ano.max()

(2000, 2021)

In [11]:
ano = df2.ano.max()
Xtreino = df2[df2.ano<ano][['diaAno', 'ano']]
ytreino = df2[df2.ano<ano]['y']
Xteste = df2[df2.ano>=ano][['diaAno', 'ano']]
yteste = df2[df2.ano>=ano]['y']

# **Modelo Auto-Sklearn**

In [12]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/content/minhaPrevisao', 
)

In [13]:
start = time()
automl.fit(Xtreino, ytreino)
end = time()
print(f"Tempo necessário para treinar o algorítmo {end - start}")

[WARNING] [2021-09-17 13:02:06,249:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] [2021-09-17 13:02:07,896:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] [2021-09-17 13:02:09,620:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] [2021-09-17 13:02:16,763:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] [2021-09-17 13:02:19,087:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] [2021-09-17 13:02:20,866:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] 

In [14]:
automl.leaderboard()

,rank,ensemble_weight,type,cost,duration
model_id,,,,,
30,1,0.42,gradient_boosting,0.000000e+00,1.009372
31,2,0.52,k_nearest_neighbors,0.000000e+00,0.853361
41,3,0.06,gaussian_process,1.210143e-14,1.259451


In [15]:
automl.show_models()

"[(0.520000, SimpleRegressionPipeline({'data_preprocessor:__choice__': 'feature_type', 'feature_preprocessor:__choice__': 'fast_ica', 'regressor:__choice__': 'k_nearest_neighbors', 'data_preprocessor:feature_type:categorical_transformer:categorical_encoding:__choice__': 'encoding', 'data_preprocessor:feature_type:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__': 'none', 'feature_preprocessor:fast_ica:algorithm': 'parallel', 'feature_preprocessor:fast_ica:fun': 'cube', 'feature_preprocessor:fast_ica:whiten': 'False', 'regressor:k_nearest_neighbors:n_neighbors': 10, 'regressor:k_nearest_neighbors:p': 2, 'regressor:k_nearest_neighbors:weights': 'uniform'},\ndataset_properties={\n  'task': 4,\n  'sparse': False,\n  'multioutput': False,\n  'target_type': 'regression',\n  'signed': False})),\n(0.42

# **Previsão**

In [16]:
label_treino = automl.predict(Xtreino)
label_teste = automl.predict(Xteste)

In [17]:
print(f"R2 score - treino: {sklearn.metrics.r2_score(ytreino, label_treino)}",
f"\nR2 score - teste: {sklearn.metrics.r2_score(yteste, label_teste)}")

R2 score - treino: 0.9999003133640553 
R2 score - teste: 1.0


# **Dataviz**

In [18]:
vdc = []
for x in Xteste.index:
  a = Xteste.loc[x]['ano']
  da = Xteste.loc[x]['diaAno']
  dc = datetime.strptime(f'{a}' + "-" + f'{da}', "%Y-%j").strftime("%d-%m-%Y")
  vdc.append(dc)

previsao = pd.DataFrame()
previsao['Realizado'] = yteste.to_list()
previsao['Previsão'] = label_teste.tolist()
previsao['Período'] = vdc

In [19]:
fig = go.Figure()
fig.add_trace(go.Bar(x=previsao['Período'], y=previsao['Realizado'],
                name='Realizado', marker_color='lightsalmon'))

fig.add_trace(go.Scatter(x=previsao['Período'], y=previsao['Previsão'],
                         mode='lines+markers',
                         name='Previsão'))
fig.update_layout(title='Comparativo de Vendas - Realizado x Previsão',
                  xaxis_title='Período',
                  yaxis_title='Total (R$)',
                  template='plotly_white',
                  xaxis = dict(tickangle=80)
                  )
fig.show()